In [1]:
from dataclasses import replace
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
import os.path
import sys
import logging
import pickle
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from models.agnn import GNNSegmentClassifier
from icecream import ic
from numpy.linalg import inv
import sklearn.metrics as metrics
from datasets import get_data_loaders
from tqdm.notebook import tqdm

In [2]:
DEVICE = "cuda:0"

In [3]:
# create model and load checkpoint
#model_result_folder = '/home1/giorgian/projects/trigger-detection-pipeline/sPHENIX/trigger_results/agnn/agnn-lr8.19806576478371e-05-b12-d71-PReLU-gi1-ln-True-n1600000/experiment_2024-04-26_13:41:37'
model_result_folder = '/home1/giorgian/projects/trigger-detection-pipeline/sPHENIX/tracking_results/agnn/agnn-lr0.003402877625756795-b32-d8-ReLU-gi1-ln-False-n1000000/experiment_2024-05-12_10:28:01/'
config_file = model_result_folder + '/config.pkl'
config = pickle.load(open(config_file, 'rb'))
data_config = config.get('data')
dphi_max, dz_max = data_config['phi_slope_max'], data_config['z0_max']

model_config = config.get('model', {})
model_config.pop('loss_func')
model_config.pop('name')
model = GNNSegmentClassifier(**model_config).to(DEVICE)

def load_checkpoint(checkpoint_file, model, optimizer=None):
    checkpoint = torch.load(checkpoint_file, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model'])
    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer'])
        return model, optimizer
    return model

# load_checkpoint
checkpoint_dir = os.path.join(model_result_folder, 'checkpoints')
checkpoint_file = sorted([os.path.join(checkpoint_dir, f) for f in os.listdir(checkpoint_dir) if f.startswith('model_checkpoint')])
checkpoint_file = checkpoint_file[-1]
print(checkpoint_file)
model = load_checkpoint(checkpoint_file, model)
print('Successfully reloaded!')

/home1/giorgian/projects/trigger-detection-pipeline/sPHENIX/tracking_results/agnn/agnn-lr0.003402877625756795-b32-d8-ReLU-gi1-ln-False-n1000000/experiment_2024-05-12_10:28:01/checkpoints/model_checkpoint_011.pth.tar
Successfully reloaded!


In [4]:
data_config

{'batch_size': 32,
 'input_dir': '/ssd3/giorgian/hits-data-march-2024/trigger/1/',
 'input_dir2': '/ssd2/giorgian/hits-data-august-2022/nontrigger/0/',
 'n_folders': 2,
 'n_mix': 12,
 'n_train': 500000,
 'n_valid': 100000,
 'n_workers': 16,
 'name': 'hit_graph',
 'phi_slope_max': 0.012193355583173944,
 'real_weight': 1.9483161321292215,
 'use_intt': True,
 'z0_max': 14.220353082111805}

In [5]:
#data_config['n_train'] = 1
data_config['n_valid'] = 100000
#data_config['input_dir2'] = '/ssd1/giorgian/hits-data-august-2022-ctypes/trigger/1'
#data_config['force_inputdir2_nontrigger'] = True

In [6]:
train_data_loader, valid_data_loader = get_data_loaders(distributed=False, rank=0, n_ranks=0, **data_config)


In [7]:
batch = next(iter(valid_data_loader))

In [8]:
batch

DataBatch(x=[24589, 5], edge_index=[2, 19269], y=[19269], w=[19269], i=[32], active_node=[24589], filename=[32], batch=[24589], ptr=[33])

In [ ]:
model.

In [17]:
pp[batch.edge_index[0]

RuntimeError: indices should be either on cpu or on the same device as the indexed tensor (cpu)

In [19]:
pred_edges, target_edges = [], []
pred_nodes, target_nodes, target_nodes_1 = [], [], []
sigmoid = torch.nn.Sigmoid()
model.eval()
for batch in tqdm(valid_data_loader):
    true_keep = batch.y.to(torch.bool)
    pred_keep = (model(batch.to(DEVICE)) > 0)
    pred_edges.extend(pred_keep.detach().cpu().numpy())
    target_edges.extend(true_keep.detach().cpu().numpy())
    pp = torch.zeros(batch.x.shape[0], dtype=bool)
    vv = torch.zeros(batch.x.shape[0], dtype=bool)
    pp[batch.edge_index[0][pred_keep]] = 1
    pp[batch.edge_index[1][pred_keep]] = 1
    vv[batch.edge_index[0][true_keep]] = 1
    vv[batch.edge_index[1][true_keep]] = 1
    pred_nodes.extend(pp.detach().cpu().numpy())
    target_nodes_1.extend(vv.detach().cpu().numpy())
    target_nodes.extend(batch.active_node.detach().cpu().numpy())

  0%|          | 0/6250 [00:00<?, ?it/s]

In [36]:
pred_edges = np.array(pred_edges)
target_edges = np.array(target_edges)
pred_nodes = np.array(pred_nodes)
target_nodes = np.array(target_nodes, dtype=bool)
target_nodes_1 = np.array(target_nodes_1, dtype=bool)

In [37]:
tp_e = np.sum(pred_edges*target_edges)
tn_e = np.sum(~pred_edges*~target_edges)
fn_e = np.sum(~pred_edges*target_edges)
fp_e = np.sum(pred_edges*~target_edges)

In [39]:
tp_n = np.sum(pred_nodes*target_nodes)
tn_n = np.sum(~pred_nodes*~target_nodes)
fn_n = np.sum(~pred_nodes*target_nodes)
fp_n = np.sum(pred_nodes*~target_nodes)

In [40]:
tp_n1 = np.sum(pred_nodes*target_nodes_1)
tn_n1 = np.sum(~pred_nodes*~target_nodes_1)
fn_n1 = np.sum(~pred_nodes*target_nodes_1)
fp_n1 = np.sum(pred_nodes*~target_nodes_1)

In [30]:
acc_e = (tp_e + tn_e)/(tp_e + tn_e + fp_e + fn_e)
prec_e = tp_e/(tp_e + fp_e)
recall_e = tp_e/(tp_e + fn_e)

In [43]:
acc_e, prec_e, recall_e

(0.9481090201418167, 0.7383537713173866, 0.8346001612381303)

In [47]:
acc_n = (tp_n + tn_n)/(tp_n + tn_n + fp_n + fn_n)
prec_n = tp_n/(tp_n + fp_n)
recall_n = tp_n/(tp_n + fn_n)

In [48]:
acc_n, prec_n, recall_n

(0.9299118820420145, 0.8033930948246946, 0.6189553260821183)

In [49]:
acc_n1 = (tp_n1 + tn_n1)/(tp_n1 + tn_n1 + fp_n1 + fn_n1)
prec_n1 = tp_n1/(tp_n1 + fp_n1)
recall_n1 = tp_n1/(tp_n1 + fn_n1)

In [50]:
acc_n1, prec_n1, recall_n1

(0.9679490989245305, 0.7646659547412282, 0.9044890971115359)